# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""795470542650947764""",6.5252e18,"""16059694790928032659""","""../../../../data/SampleHDS.jso…","""14152965782273010791""","""17657698819028080637""",false
"""7747942690140083009""",1.2602e18,"""5678168896995548438""","""../../../../data/SampleHDS.jso…","""14152965782273010791""","""9418057028760544756""",true
"""13909698919238809691""",1.3990e19,"""4490851485659337937""","""../../../../data/SampleHDS.jso…","""14729588571204805445""","""17657698819028080637""",false
"""18022688109823518762""",3.6987e18,"""5678168896995548438""","""../../../../data/SampleHDS.jso…","""8439336320454046961""","""17657698819028080637""",false
"""4708720033607670321""",1.5620e19,"""16059694790928032659""","""../../../../data/SampleHDS.jso…","""8439336320454046961""","""1249143407491477972""",true
"""4210289601518336501""",8.3466e18,"""4490851485659337937""","""../../../../data/SampleHDS.jso…","""18434988546458978529""",null,false
"""15604252109584585756""",2.1037e18,"""4490851485659337937""","""../../../../data/SampleHDS.jso…","""8439336320454046961""","""1249143407491477972""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""2742402609161262927""",3.4430e18,"""9291639150031189430""","""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""419096811171409744""","""Rejected"""
"""12223096994811160508""",4.2669e18,"""13738449543083054085""","""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""6023664356922739631""","""Accepted"""
"""13381497734241155053""",1.6401e19,"""6778766994719090455""","""../../../../data/SampleHDS.jso…","""MoneyMarketSavingsAccount""","""419096811171409744""","""Rejected"""
"""2646088907403870476""",6.5243e18,"""13738449543083054085""","""../../../../data/SampleHDS.jso…","""BasicChecking""","""419096811171409744""","""Rejected"""
"""1559000332094691741""",1.0095e18,"""9291639150031189430""","""../../../../data/SampleHDS.jso…","""BasicChecking""","""6010183960746362401""","""Accepted"""
"""11694959838813486563""",3.5195e18,"""6778766994719090455""","""../../../../data/SampleHDS.jso…","""UPlusFinPersonal""",null,"""Rejected"""
"""14498360098373855464""",6.9678e18,"""6778766994719090455""","""../../../../data/SampleHDS.jso…","""BasicChecking""","""6010183960746362401""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""2367453110802760380""",1.5330e19,"""9335179623399248105""","""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""2375532983593433660""","""Rejected"""
"""16370374282811519483""",1.4982e19,"""8921306619331836235""","""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""12652014746904830672""","""Accepted"""
"""10776462071605567053""",7.4551e18,"""1095654781442232193""","""../../../../data/SampleHDS.jso…","""MoneyMarketSavingsAccount""","""2375532983593433660""","""Rejected"""
"""1311660277637660253""",8.5549e18,"""8921306619331836235""","""../../../../data/SampleHDS.jso…","""BasicChecking""","""2375532983593433660""","""Rejected"""
"""5339408134475176182""",6.4534e18,"""9335179623399248105""","""../../../../data/SampleHDS.jso…","""BasicChecking""","""7051141762435135260""","""Accepted"""
"""14514825350656967983""",9.1316e18,"""1095654781442232193""","""../../../../data/SampleHDS.jso…","""UPlusFinPersonal""",null,"""Rejected"""
"""16526616692047699199""",9.4323e18,"""1095654781442232193""","""../../../../data/SampleHDS.jso…","""BasicChecking""","""7051141762435135260""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
Customer_CLV=PREDICTOR_1
Customer_MaritalStatus=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""15370721970324918795""",7.7396e18,"""13758520771433417676""","""FirstMortgage30yr""","""11536232935238918781""","""Rejected"""
"""18382913153702564300""",1.4392e19,"""1931693483292258967""","""FirstMortgage30yr""","""3057198860239785687""","""Accepted"""
"""5603129501703082898""",8.1198e18,"""6767418375704386890""","""MoneyMarketSavingsAccount""","""11536232935238918781""","""Rejected"""
"""8971610426504297500""",7.8131e18,"""1931693483292258967""","""BasicChecking""","""11536232935238918781""","""Rejected"""
"""14114570098984104195""",1.2082e19,"""13758520771433417676""","""BasicChecking""","""2948221747051975111""","""Accepted"""
"""17535626493183681514""",1.7870e19,"""6767418375704386890""","""UPlusFinPersonal""",null,"""Rejected"""
"""10502679744977247574""",1.2575e19,"""6767418375704386890""","""BasicChecking""","""2948221747051975111""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'